**Import Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


**Load and Preprocess Data**

In [2]:
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape((-1, 28, 28, 1))
    x_test = x_test.reshape((-1, 28, 28, 1))
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_and_preprocess_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Build CNN Model**

In [3]:
def build_cnn_model(num_filters1=32, num_filters2=64, kernel_size=(3, 3), dense_units=128):
    model = models.Sequential()
    model.add(layers.Conv2D(num_filters1, kernel_size, activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(num_filters2, kernel_size, activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model


**Compile Model**

In [4]:
def compile_model(model, learning_rate=0.001):
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


**Train and Evaluate Model**

In [8]:
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, batch_size, epochs):
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=0)
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return test_loss, test_accuracy


**Hyperparameter Tuning**

In [9]:
def plot_training_results(history):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


**Model Summary for Best Hyperparameters**

In [11]:
best_accuracy = 0
best_params = {}

hyperparams_combinations = [
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 128, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 16, 'num_filters2': 32, 'kernel_size': (3, 3), 'dense_units': 64, 'learning_rate': 0.002, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 128, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 16, 'num_filters2': 32, 'kernel_size': (5, 5), 'dense_units': 64, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 64, 'learning_rate': 0.002, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 128, 'num_filters2': 256, 'kernel_size': (3, 3), 'dense_units': 512, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5}
]

for params in hyperparams_combinations:
    print(f"Training with parameters: {params}")
    model = build_cnn_model(params['num_filters1'], params['num_filters2'], params['kernel_size'], params['dense_units'])
    model = compile_model(model, params['learning_rate'])
    test_loss, test_accuracy = train_and_evaluate_model(model, x_train, y_train, x_test, y_test, params['batch_size'], params['epochs'])
    print(f"Accuracy: {test_accuracy:.4f}, Loss: {test_loss:.4f}\n")

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_params = params

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.4f}")


Training with parameters: {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy: 0.9908, Loss: 0.0255

Training with parameters: {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 128, 'epochs': 5}
Accuracy: 0.9926, Loss: 0.0230

Training with parameters: {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5}
Accuracy: 0.9860, Loss: 0.0419

Training with parameters: {'num_filters1': 16, 'num_filters2': 32, 'kernel_size': (3, 3), 'dense_units': 64, 'learning_rate': 0.002, 'batch_size': 64, 'epochs': 5}
Accuracy: 0.9900, Loss: 0.0320

Training with parameters: {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 128, 'epochs': 5}
Accuracy: 0.9937, Loss: 0.0219

Training with parameters: {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 5}
Accuracy:

**Summary of the best model**

In [12]:
# Train the model one more time with best parameters for visualization and summary
print(f"Best Hyperparameters: {best_params}")

best_model = build_cnn_model(
    best_params['num_filters1'],
    best_params['num_filters2'],
    best_params['kernel_size'],
    best_params['dense_units']
)
best_model = compile_model(best_model, best_params['learning_rate'])
train_and_evaluate_model(
    best_model, x_train, y_train, x_test, y_test, best_params['batch_size'], best_params['epochs']
)

# Display the model summary
best_model.summary()


Best Hyperparameters: {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 128, 'epochs': 5}


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)                   │ (None, 24, 24, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_28 (MaxPooling2D)      │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 8, 8, 128)           │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_29 (MaxPooling2D)      │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_14 (Flatten)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,410,464 (5.38 MB)

 Trainable params: 470,154 (1.79 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 940,310 (3.59 MB)

In [13]:
best_model.save('mnist_2.h5')